In [1]:
import pandas as pd
import requests
import zipfile
import io
from config import URL_CONTIGUI

In [4]:
response = requests.get(URL_CONTIGUI)
if response.status_code != 200:
    raise Exception("Failed to download file")

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    # Step 3: Find and read all .xlsx files
    dataframes = []
    for file_name in z.namelist():
        if file_name.endswith(".xlsx"):
            with z.open(file_name) as f:
                df = pd.read_excel(f, dtype=str)
                df["source_file"] = file_name
                dataframes.append(df)

final_df = pd.concat(dataframes, ignore_index=True)

print(final_df.head())

/Users/rauldemaio/Projects Local/graph_nn_cookbook/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/rauldemaio/Projects Local/graph_nn_cookbook/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/rauldemaio/Projects Local/graph_nn_cookbook/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/rauldemaio/Projects Local/graph_nn_cookbook/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or foote

  COD_REG COD_PRO COMUNE COD_PRO COMUNE ADIACENTE PRO_COM COMUNE  \
0       1              1                        1           1001   
1       1              1                        1           1001   
2       1              1                        1           1001   
3       1              1                        1           1001   
4       1              1                        1           1001   

  PRO_COM COMUNE ADIACENTE COMUNE      COMUNE ADIACENTE   LUNGHEZZA CONFINE  \
0                     1015  Agliè                 Bairo  3957.1980647043697   
1                     1096  Agliè              Cuceglio   2763.288258827889   
2                     1176  Agliè                Ozegna  2936.9684103524646   
3                     1244  Agliè  San Giorgio Canavese  2689.6178637264816   
4                     1247  Agliè  San Martino Canavese  1382.2900835216005   

  LUNGHEZZA CONFINE.1                                        source_file  \
0    3.95719806470437  MatriciContiguita

/Users/rauldemaio/Projects Local/graph_nn_cookbook/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [6]:
# redefine column COD_REG as COL_REG if COL_REG COMUNE is nan, otherwise keep COL_REG COMUNE
final_df["COD_REG"] = final_df.apply(
    lambda row: row["COD_REG COMUNE"] if pd.notna(row["COD_REG COMUNE"]) else row["COD_REG"], axis=1
)
final_df["LUNGHEZZA CONFINE KM"] = final_df.apply(
    lambda row: row["LUNGHEZZA CONFINE KM"] if pd.notna(row["LUNGHEZZA CONFINE KM"]) else row["LUNGHEZZA CONFINE.1"], axis=1
)
final_df["COD_PRO COMUNE ADIACENTE"] = final_df.apply(
    lambda row: row["COD_PROV COMUNE ADIACENTE"] if pd.notna(row["COD_PROV COMUNE ADIACENTE"]) else row["COD_PRO COMUNE ADIACENTE"], axis=1
)

In [11]:
final_df = final_df.iloc[:,:-3]
final_df.columns = final_df.columns.str.replace(" ", "_")\
    .str.lower()\
        .str.strip()

In [14]:
final_df

,cod_reg,cod_pro_comune,cod_pro_comune_adiacente,pro_com_comune,pro_com_comune_adiacente,comune,comune_adiacente
0,1,1,1,1001,1015,Agliè,Bairo
1,1,1,1,1001,1096,Agliè,Cuceglio
2,1,1,1,1001,1176,Agliè,Ozegna
3,1,1,1,1001,1244,Agliè,San Giorgio Canavese
4,1,1,1,1001,1247,Agliè,San Martino Canavese
...,...,...,...,...,...,...,...
232343,20,111,111,111106,111096,Villasor,Villacidro
232344,20,111,92,111107,92015,Villaspeciosa,Decimomannu
232345,20,111,92,111107,92090,Villaspeciosa,Uta
232346,20,111,111,111107,111013,Villaspeciosa,Decimoputzu


In [15]:
final_df.to_parquet("../../data/02_primary/matrice_contigui.parquet", index=False)